In [ ]:
!pip install transformers
!pip install pandas
!pip install arabert


import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from arabert.preprocess import ArabertPreprocessor
#Load AraBERT model and tokenizer
model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load dataset
df = pd.read_csv('ar_reviews_100k.tsv', delimiter='\t', nrows=3000)

# Initialize ArabertPreprocessor for cleaning the text
arabert_prep = ArabertPreprocessor(model_name=model_name)

# Tokenize text and get sentiment predictions
sentiment_labels = []
for text in df['text']:
    cleaned_text = arabert_prep.preprocess(text)
    inputs = tokenizer.encode_plus(cleaned_text, return_tensors="pt", add_special_tokens=True, max_length=512, truncation=True)
    outputs = model(inputs['input_ids'], token_type_ids=None, attention_mask=inputs['attention_mask'])
    sentiment = torch.argmax(outputs[0]).item()
    sentiment_labels.append(sentiment)

# Calculate percentage of each sentiment class in the dataset
num_positive = sum([1 for label in sentiment_labels if label == 1])
num_negative = sum([1 for label in sentiment_labels if label == 0])
num_neutral = sum([1 for label in sentiment_labels if label == 2])
total = len(sentiment_labels)
percent_positive = (num_positive / total) * 100
percent_negative = (num_negative / total) * 100
percent_neutral = (num_neutral / total) * 100

# Print results
print("Positive: {:.2f}%".format(percent_positive))
print("Negative: {:.2f}%".format(percent_negative))
print("Neutral: {:.2f}%".format(percent_neutral))